# ConversationKGMemory

- Author: [Secludor](https://github.com/Secludor)
- Design: [Secludor](https://github.com/Secludor)
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## Overview

Unlike Entity Memory, which manages information about entities in a key-value format for individual entities, `Conversation Knowledge Graph Memory` is a module that manages relationships between entities in a graph format.

It extracts and structures **knowledge triplets** (subject-relationship-object) to identify and store complex relationships between entities, and allows exploration of entity connectivity through **graph structure**.

This helps the model understand relationships between different entities and better respond to queries based on complex networks and historical context.

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [Conversation Knowlege Graph Memory](#conversation-knowlege-graph-memory)
- [Apply to Chain](#apply-to-chain)

### References

- [LangChain Python API Reference>langchain-community: 0.3.13>memory>ConversationKGMemory](https://python.langchain.com/api_reference/community/memory/langchain_community.memory.kg.ConversationKGMemory.html)
----

## Environment Setup
[return](#overview)

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_community",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "05-ConversationKGMemory",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Conversation Knowlege Graph Memory
[return](#overview)

`Conversation Knowledge Graph Memory` is a memory module that stores and manages information extracted from conversations in a graph structure. This example demonstrates the following key features:

- Storing conversation context (`save_context`)
- Extracting entities from current conversation (`get_current_entities`)
- Extracting knowledge triplets (`get_knowledge_triplets`)
- Retrieving stored memory (`load_memory_variables`)

The following example shows the process of extracting entities and relationships from a conversation about a new designer, Shelly Kim, and storing them in a graph format.

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.memory.kg import ConversationKGMemory

In [6]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "This is Shelly Kim who lives in Pangyo."},
    {"output": "Hello Shelly, nice to meet you! What kind of work do you do?"},
)
memory.save_context(
    {"input": "Shelly Kim is our company's new designer."},
    {
        "output": "That's great! Welcome to our team. I hope you'll enjoy working with us."
    },
)

In [7]:
memory.get_current_entities({"input": "Who is Shelly Kim?"})

['Shelly Kim']

In [8]:
memory.get_knowledge_triplets({"input": "Shelly"}), memory.get_knowledge_triplets(
    {"input": "Pangyo"}
), memory.get_knowledge_triplets({"input": "job"})

([KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo'),
  KnowledgeTriple(subject='Shelly Kim', predicate='is', object_="company's new designer")],
 [KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')],
 [KnowledgeTriple(subject='Shelly Kim', predicate='is a', object_='designer'),
  KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')])

In [9]:
memory.load_memory_variables({"input": "Who is Shelly Kim?"})

{'history': [SystemMessage(content="On Shelly Kim: Shelly Kim lives in Pangyo. Shelly Kim is our company's new designer.", additional_kwargs={}, response_metadata={})]}

## Apply to Chain
[return](#overview)

Let's examine the memory after having a conversation using ConversationChain with ConversationKGMemory set as its memory.


In [10]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

C:\Users\Caelu\AppData\Local\Temp\ipykernel_55328\3781685186.py:20: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_with_kg = ConversationChain(


Let's start the first conversation by providing some basic information about a person.

In [11]:
conversation_with_kg.predict(
    input="My name is Teddy. Shelly is a coworker of mine, and she's a new designer at our company."
)

"Hi Teddy! It's great to meet you. It sounds like Shelly is embarking on an exciting journey as a new designer at your company. Design roles can be quite dynamic and creative, offering opportunities to work on a variety of projects. Is there anything specific about her role or the projects she's working on that you'd like to share or discuss?"

Let's proceed with a question about Shelly.

In [12]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': 'On Shelly: Shelly is a coworker of Teddy. Shelly is a new designer. Shelly works at our company.'}

You can also reset the memory by `memory.clear()`.

In [13]:
conversation_with_kg.memory.clear()
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': ''}